# 数据处理

In [ ]:
import os
os.environ["http_proxy"] = "http://127.0.0.1:1083"
os.environ["https_proxy"] = "http://127.0.0.1:1083"

! pip install langchain_community
! pip install langchain
! pip install transformers
! pip install unstructured
! pip install markdown
! pip install markdownify
! pip install sentence-transformers
! pip install clickhouse-connect
! pip install -U langchain_qdrant



# embedding 测试

In [1]:
import os
os.environ["http_proxy"] = "http://127.0.0.1:1083"
os.environ["https_proxy"] = "http://127.0.0.1:1083"

from sentence_transformers import SentenceTransformer

model = SentenceTransformer('moka-ai/m3e-base')

#Our sentences we like to encode
sentences = [
    'hi',
]

#Sentences are encoded by calling model.encode()
embeddings = model.encode(sentences)

#Print the embeddings
for sentence, embedding in zip(sentences, embeddings):
    print("Sentence:", sentence)
    # print("Embedding:", embedding)
    print("Embedding:", embedding)
    print("")

/root/aiprojects/rag_demo/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/root/aiprojects/rag_demo/.venv/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/root/aiprojects/rag_demo/.venv/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Sentence: hi
Embedding: [ 1.15814638e+00  1.07707655e+00  7.56560802e-01 -9.45001781e-01
 -6.23663008e-01 -5.91837525e-01 -5.17009318e-01  3.37781876e-01
 -7.85273075e-01  3.03312778e-01  1.71803713e-01 -2.45471075e-01
  4.48939413e-01 -7.74856627e-01 -1.47187519e+00 -3.84476036e-02
 -1.97529331e-01  4.16731834e-01 -1.08227694e+00 -6.37392700e-01
  3.76232147e-01  2.90069818e-01 -9.65722561e-01  3.14127654e-01
 -8.34281668e-02 -8.29551220e-01 -8.29409361e-01  6.41668200e-01
  9.15428877e-01  1.44395083e-01  4.72518921e-01  5.36171310e-02
 -1.24945797e-01  3.62783551e-01  6.10816896e-01 -6.12878613e-02
  1.03877485e+00 -4.88635868e-01 -4.16323423e-01  3.16369444e-01
  2.18434885e-01  9.69376266e-01 -3.96862358e-01 -5.03182232e-01
 -5.29303849e-01  8.70548561e-02  7.38626182e-01  5.42922802e-02
 -1.06339180e+00 -1.30282924e-01  6.46417558e-01  5.54705477e+00
  2.18839422e-01  4.34251904e-01 -5.92861712e-01  6.75608814e-01
 -5.08189142e-01 -2.98218220e-01  1.15736179e-01 -3.26271772e-01
 

In [18]:
import os
os.environ["http_proxy"] = "http://127.0.0.1:1083"
os.environ["https_proxy"] = "http://127.0.0.1:1083"

from langchain.embeddings import HuggingFaceEmbeddings

model_kwargs = {"device": "cuda"}
embeddings = HuggingFaceEmbeddings(
    model_name="moka-ai/m3e-large", model_kwargs=model_kwargs)

# print(embeddings.embed_query("hello"))
print(embeddings.encode_kwargs)
# print(embeddings.embed_query("hello"))

No sentence-transformers model found with name moka-ai/m3e-large. Creating a new one with MEAN pooling.
/root/aiprojects/rag_demo/.venv/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


{}


## 查看目录中文档的最大长度

In [ ]:
import os

root_dir = "/root/jd_docs"
size = 0
for root, dirs, files in os.walk(root_dir, followlinks=False):
    for file in files:
        path = os.path.join(root, file)
        if os.path.getsize(path) > size:
            size = os.path.getsize(path)
size

## 查看目录中json content字段最大长度

In [ ]:
import os
import json

root_dir = "/root/jd_docs"
size = 0
for root, dirs, files in os.walk(root_dir, followlinks=False):
    for file in files:
        path = os.path.join(root, file)
        with open(path, 'r') as docfile:
            doc_data = json.load(docfile)
            if size < len(doc_data["content"]):
                print(type(doc_data["content"]))
                size = len(doc_data["content"])

size

## 去标签实验

In [ ]:
from markdownify import markdownify as md
import re


def is_html(content):
    # content = content.read(15)  # 读取文件开头的15个字节
    if content.startswith('<'):
        return True
    else:
        return False


def replace_unicode(match):
    code_point = int(match.group(1), 16)
    return chr(code_point)


# f = open("/root/jd_docs/account-assets/withdrawal.md")
f = open("/root/jd_docs/application-load-balancer/TLSSecurityPolicy-management.md")
lines = f.read()


# 替换unicode字符为标签
result = re.sub(r"\\[uU]([0-9a-fA-F]{4})", replace_unicode, lines)


# 如果为html 格式，转换为markdown
if is_html(result):
    result = md(result)
    print(result)

# 去html标签
pattern = re.compile(r'<[^>]+>', re.S)
result = pattern.sub(' ', result)

print(result.replace('\\t', '').replace('\\n', ''))
f.close()

## 文本转markdown

In [ ]:
from markdownify import markdownify

markdownify("要对 MySQL 进行性能测试，可以按照以下步骤进行：1. **准备环境**：   - 安装必要的工具，如 `sysbench`，用于执行性能测试。   - 创建云数据库实例，如华北-北京可用区A的 MySQL 5.7 服务器。   - 选择合适的云主机规格和镜像，如 8C 32GB 的 CentOS 7.4 64位。   - 为数据库实例创建一个名为 `sbtest` 的库，并为创建的账号授予新建库的读写权限。   - 使用 `sysbench` 创建一个包含表结构、数据格式和 SQL 样式的测试样本。2. **测试环境设置**：   - 在 `sysbench` 中，设置数据库实例的参数，如最大连接数（最大 IOPS）、表大小、数据库驱动、MySQL 服务器地址、用户名、密码等。   - 设置测试用例的参数，如表结构、数据量大小、客户端数量、查询类型（如 get、set、lpush、mset 等）、查询时间（如 1800 秒）和并发数（如 32 个线程）。3. **性能压测**：   - 使用 `sysbench` 创建一个包含表结构、数据格式和 SQL 样式的测试样本，并设置测试参数。   - 使用 `sysbench` 进行性能压测，包括查询、更新和删除操作，以生成数据负载并进行压力测试。   - 使用 `sysbench` 输出查询时间、并发数、TPS 和 QPS 的统计信息，以评估数据库性能。4. **环境清理**：   - 清理测试环境，包括删除测试样本、关闭数据库实例、删除 Redis 服务器等。5. **性能指标测试**：   - 使用 `redis-benchmark` 测试 Redis 性能，包括获取、设置、LPush、MSet（10 keys）、SADD 等操作的性能指标。   - 根据测试结果，评估 Redis 性能瓶颈，如查询时间、并发数、TPS 和 QPS 的瓶颈，以及数据库性能瓶颈。通过以上步骤，可以对 MySQL 进行性能测试，评估其在高并发、大量数据和复杂查询场景下的性能表现，为优化 MySQL 服务器配置和性能优化提供参考。")

## markdown to html


In [ ]:
import markdown

text = """在实例中创建账号和密码，用户可以通过该账号连接到 Starwift 中。操作步骤如下：\n\n1. 进入【账号管理】页面：在控制台中，点击实例名，进入实例页面，选择 **账号管理** 页面。\n2. 创建账号：点击 **创建账号**，弹窗的提示信息，输入 **账号名称、密码及权限** 信息，并点击 **确定** 完成账号创建。\n\n创建账号\n----\n\n\n在实例中创建账号和密码，用户可以通过该账号连接到 Starwift 中。操作步骤如下：\n\n1. 进入【账号管理】页面：在控制台中，点击实例名，进入实例页面，选择 **账号管理** 页面。\n2. 创建账号：点击 **创建账号**，弹窗的提示信息，输入 **账号名称、密码及权限** 信息，并点击 **确定** 完成账号创建。\n\n新建实例\n----\n\n\n在控制台上新创建一个 Starwift 实例。操作步骤如下：\n\n1. 进入实例列表页：当前页面会显示当前区域下的各个实例的概要信息，包括计费信息和过期时间等。点击 **创建** 按钮，新建一个 Starwift 实例。\n\n\n[![\\\"实例列表\\\"](\"https://jdcloud-portal.oss-cn-north-1.jcloudcs.com/cn/image/Starwift/instance-list.png\")](\"https://jdcloud-portal.oss-cn-north-1.jcloudcs.com/cn/image/Starwift/instance-list.png\")\n\n#### 2. 选择计费方式及地域：\n   - 在数据库配置页面，选择计费方式（包年包月或按配置），地域，可看到当前地域上CPU核数的配额。 用户创建的实例的总核数不能超过当前地域下的剩余配额。 如果配额不足，可通过工单申请提升。\n\n#### 3. 配置节点规格及数目：\n   - 设置计算节点的规格数目。\n   - 设置 ZooKeeper 节点的规格和数目。 通常情况下3个 ZK 节点即可满足绝大部分场景的需求。\n   - 设置监控节点的规格。 监控节点用于收集和上报集群的监控信息。每个实例只需要1个监控节点。\n\n\n[![\\\"实例配置1\\\"](\"https://jdcloud-portal.oss-cn-north-1.jcloudcs.com/cn/image/Starwift/create-instance-2.png\")](\"https://jdcloud-portal.oss-cn-north-1.jcloudcs.com/cn/image/Starwift/create-instance-2.png\")\n\n#### 4. 配置 网络，可用区及实例名称：\n   - Starwift 只支持在私有网络中创建。如果用户没有私有网络及子网，可以通过 **“新建私有网络”** 和 **“新建子网”** 的链接创建私有网络和子网。创建完成后，点击 **“刷新”**，就可以看到新创建的私有网络和子网了。\n   - 设置实例部署的可用区。目前 Starwift 暂时仅支持单可用区部署。\n   - 输入实例名称：\n      - [示例]：`MyStarwiftInstance` 或 `my-starwift-instance` （根据实际需求填写）\n\n#### 5. 确认购买：\n   - 信息输入完成后，可查看点击页面右边的价格信息，并点击 **立即购买**。 在订单确认页面，确认购买信息无误后，点击 **立即支付**。 支付成功后自动返回实例列表页面，能看到实例状态为 **“创建中”**。稍等一会后，实例创建完成，显示 **“运行”** 中\n\n以上就是在实例中创建账号和密码的基本步骤，具体操作可能因不同版本的 Starwift 版本和环境而略有差异。请根据您使用的具体版本和环境进行相应的调整。"""
markdown.markdown(text)

## markdown to text


In [ ]:
import markdown  # pip install markdown
from bs4 import BeautifulSoup  # pip install beautifulsoup4


def md_to_text(md):
    html = markdown.markdown(md)
    soup = BeautifulSoup(html, features='html.parser')
    return soup.get_text()


text = """在实例中创建账号和密码，用户可以通过该账号连接到 Starwift 中。操作步骤如下：\n\n1. 进入【账号管理】页面：在控制台中，点击实例名，进入实例页面，选择 **账号管理** 页面。\n2. 创建账号：点击 **创建账号**，弹窗的提示信息，输入 **账号名称、密码及权限** 信息，并点击 **确定** 完成账号创建。\n\n创建账号\n----\n\n\n在实例中创建账号和密码，用户可以通过该账号连接到 Starwift 中。操作步骤如下：\n\n1. 进入【账号管理】页面：在控制台中，点击实例名，进入实例页面，选择 **账号管理** 页面。\n2. 创建账号：点击 **创建账号**，弹窗的提示信息，输入 **账号名称、密码及权限** 信息，并点击 **确定** 完成账号创建。\n\n新建实例\n----\n\n\n在控制台上新创建一个 Starwift 实例。操作步骤如下：\n\n1. 进入实例列表页：当前页面会显示当前区域下的各个实例的概要信息，包括计费信息和过期时间等。点击 **创建** 按钮，新建一个 Starwift 实例。\n\n\n[![\\\"实例列表\\\"](\"https://jdcloud-portal.oss-cn-north-1.jcloudcs.com/cn/image/Starwift/instance-list.png\")](\"https://jdcloud-portal.oss-cn-north-1.jcloudcs.com/cn/image/Starwift/instance-list.png\")\n\n#### 2. 选择计费方式及地域：\n   - 在数据库配置页面，选择计费方式（包年包月或按配置），地域，可看到当前地域上CPU核数的配额。 用户创建的实例的总核数不能超过当前地域下的剩余配额。 如果配额不足，可通过工单申请提升。\n\n#### 3. 配置节点规格及数目：\n   - 设置计算节点的规格数目。\n   - 设置 ZooKeeper 节点的规格和数目。 通常情况下3个 ZK 节点即可满足绝大部分场景的需求。\n   - 设置监控节点的规格。 监控节点用于收集和上报集群的监控信息。每个实例只需要1个监控节点。\n\n\n[![\\\"实例配置1\\\"](\"https://jdcloud-portal.oss-cn-north-1.jcloudcs.com/cn/image/Starwift/create-instance-2.png\")](\"https://jdcloud-portal.oss-cn-north-1.jcloudcs.com/cn/image/Starwift/create-instance-2.png\")\n\n#### 4. 配置 网络，可用区及实例名称：\n   - Starwift 只支持在私有网络中创建。如果用户没有私有网络及子网，可以通过 **“新建私有网络”** 和 **“新建子网”** 的链接创建私有网络和子网。创建完成后，点击 **“刷新”**，就可以看到新创建的私有网络和子网了。\n   - 设置实例部署的可用区。目前 Starwift 暂时仅支持单可用区部署。\n   - 输入实例名称：\n      - [示例]：`MyStarwiftInstance` 或 `my-starwift-instance` （根据实际需求填写）\n\n#### 5. 确认购买：\n   - 信息输入完成后，可查看点击页面右边的价格信息，并点击 **立即购买**。 在订单确认页面，确认购买信息无误后，点击 **立即支付**。 支付成功后自动返回实例列表页面，能看到实例状态为 **“创建中”**。稍等一会后，实例创建完成，显示 **“运行”** 中\n\n以上就是在实例中创建账号和密码的基本步骤，具体操作可能因不同版本的 Starwift 版本和环境而略有差异。请根据您使用的具体版本和环境进行相应的调整。"""

# print(md_to_text(text))
t = md_to_text(text)
print(t)

## 文本分割，拆分为定长文本


In [ ]:
from langchain.text_splitter import CharacterTextSplitter
from langchain_community.document_loaders import UnstructuredMarkdownLoader
from langchain.document_loaders import TextLoader
from langchain_community.document_loaders import DirectoryLoader
import os
os.environ["http_proxy"] = "http://127.0.0.1:1083"
os.environ["https_proxy"] = "http://127.0.0.1:1083"

root_dir = "/root/jd_docs"

loader = DirectoryLoader(
    '/root/jd_docs', glob="**/*.json", loader_cls=TextLoader)
docs = loader.load()

for new_doc in docs[0:2]:
    print(new_doc)

## 解析自定义json到docs

In [ ]:
# 加载自定义模块路径
import sys
sys.path.append("..")

In [ ]:
from libs.jd_doc_json_loader import JD_DOC_Loader
from langchain.text_splitter import CharacterTextSplitter
from langchain_community.document_loaders import UnstructuredMarkdownLoader
from langchain_community.document_loaders import DirectoryLoader


root_dir = "/root/jd_docs"
loader = DirectoryLoader(
    '/root/jd_docs', glob="**/*.json", loader_cls=JD_DOC_Loader)
docs = loader.load()
print(docs[0:2])

### splite doc by MarkdownHeaderTextSplitter

In [ ]:
from langchain.text_splitter import MarkdownHeaderTextSplitter, MarkdownTextSplitter
headers_to_split_on = [
    ("#", "#"),
    ("##", "##"),
    ("###", "###"),
    ("####", "####"),
    ("#####", "#####"),
]
MarkdownTextSplitter
markdown_splitter = MarkdownHeaderTextSplitter(
    headers_to_split_on=headers_to_split_on)

count = 0
for doc in docs:
    chunked_docs = markdown_splitter.split_text(doc.page_content)
    head = doc.metadata["product"]+"--"+doc.metadata["title"]+"\n"
    for chunded_doc in chunked_docs:
        doc_head = head
        if len(chunded_doc.metadata.items()) > 0:
            for k, v in chunded_doc.metadata.items():
                doc_head = doc_head+k+v+"\n"

        if len(head) < len(doc_head):
            title_doc = doc
            title_doc.page_content = doc_head
            print(title_doc)

        content = doc_head+chunded_doc.page_content
        new_doc = doc
        new_doc.page_content = content
        print(new_doc)
        print("-------------------------------------------")

## 存入clickhouse向量数据库

In [ ]:
# 加载自定义模块路径
import sys
sys.path.append("..")

In [ ]:
from typing import List
from langchain.text_splitter import MarkdownHeaderTextSplitter, MarkdownTextSplitter
import langchain_community.vectorstores.clickhouse as clickhouse
from langchain.embeddings import HuggingFaceEmbeddings
from langchain_community.document_loaders import DirectoryLoader
from libs.jd_doc_json_loader import JD_DOC_Loader

root_dir = "/root/jd_docs"
loader = DirectoryLoader(
    '/root/jd_docs', glob="**/*.json", loader_cls=JD_DOC_Loader)
docs = loader.load()
model_kwargs = {"device": "cuda"}
embeddings = HuggingFaceEmbeddings(
    model_name="/root/models/moka-ai-m3e-large", model_kwargs=model_kwargs)

headers_to_split_on = [
    ("#", "#"),
    ("##", "##"),
    ("###", "###"),
    ("####", "####"),
    ("#####", "#####"),
]
MarkdownTextSplitter
markdown_splitter = MarkdownHeaderTextSplitter(
    headers_to_split_on=headers_to_split_on)

new_docs = []
batch = 200

settings = clickhouse.ClickhouseSettings(
    table="jd_docs_m3e_with_url_splited", username="default", password="Git785230", host="10.0.1.94")

count = 0
# 将 product tilte 作为正文加入document.page_content,分批次入库
for doc in docs:
    chunked_docs = markdown_splitter.split_text(doc.page_content)
    head = doc.metadata["product"]+"--"+doc.metadata["title"]+"\n"
    for chunded_doc in chunked_docs:
        doc_head = head
        if len(chunded_doc.metadata.items()) > 0:
            for k, v in chunded_doc.metadata.items():
                doc_head = doc_head+k+v+"\n"

        if len(head) < len(doc_head):
            title_doc = doc
            title_doc.page_content = doc_head
            count += 1
            new_docs.append(title_doc)

        content = doc_head+chunded_doc.page_content
        new_doc = doc
        new_doc.page_content = content
        count += 1
        new_docs.append(new_doc)

        if len(new_docs) > batch:
            _ = clickhouse.Clickhouse.from_documents(
                new_docs, embeddings, config=settings)
            new_docs.clear()
if len(new_docs) > 0:
    _ = clickhouse.Clickhouse.from_documents(
        new_docs, embeddings, config=settings)
print(count)

In [ ]:
import langchain_community.vectorstores.clickhouse as clickhouse
from langchain.embeddings import HuggingFaceEmbeddings
import os
os.environ["http_proxy"] = "http://127.0.0.1:1083"
os.environ["https_proxy"] = "http://127.0.0.1:1083"
model_kwargs = {"device": "cuda"}
embeddings = HuggingFaceEmbeddings(
    model_name="/root/models/moka-ai-m3e-large", model_kwargs=model_kwargs)

for doc in docs:
    t = embeddings.embed_documents(doc.page_content)
    print(t)


settings = clickhouse.ClickhouseSettings(
    table="jd_docs_m3e_with_url_splited", username="default", password="Git785230", host="10.0.1.94")


# docsearch = clickhouse.Clickhouse.from_documents(
#     docs, embeddings, config=settings)

In [ ]:
import langchain_community.vectorstores.clickhouse as clickhouse
from langchain.embeddings import HuggingFaceEmbeddings

model_kwargs = {"device": "cuda"}
embeddings = HuggingFaceEmbeddings(
    model_name="/root/models/moka-ai-m3e-large", model_kwargs=model_kwargs)

settings = clickhouse.ClickhouseSettings(
    table="jd_docs_m3e_with_url_splited", username="default", password="Git785230", host="10.0.1.94")

docsearch = clickhouse.Clickhouse.from_documents(
    docs, embeddings, config=settings)

## 验证ck向量库

In [ ]:
import langchain_community.vectorstores.clickhouse as clickhouse
from langchain.embeddings import HuggingFaceEmbeddings

model_kwargs = {"device": "cuda"}
embeddings = HuggingFaceEmbeddings(
    model_name="/root/models/moka-ai-m3e-large", model_kwargs=model_kwargs)

settings = clickhouse.ClickhouseSettings(
    table="jd_docs_m3e_with_url_splited", username="default", password="Git785230", host="10.0.1.94")
ck_db = clickhouse.Clickhouse(embeddings, config=settings)

ck_db._build_query_sql
ck_retriever = ck_db.as_retriever(
    search_type="similarity_score_threshold", search_kwargs={'score_threshold': 0.9})
ck_retriever.get_relevant_documents("如何创建mysql rds")

In [ ]:
from langchain.embeddings import HuggingFaceEmbeddings
import clickhouse_connect

model_kwargs = {"device": "cuda"}
embeddings = HuggingFaceEmbeddings(
    model_name="/root/models/moka-ai-m3e-large", model_kwargs=model_kwargs)
# v = embeddings.embed_query("如何创建对象存储的bucket")
# v = embeddings.embed_query("Massive 此文本嵌入模型通过**千万级**的中文句对数据集进行训练")
v = embeddings.embed_query("对象存储api如何使用")
q_emb_str = ",".join(map(str, v))

client = clickhouse_connect.get_client(
    host='10.0.1.94', username='default', password='Git785230')
q_str = f"""
        SELECT document,metadata,dist
        FROM default.jd_docs_m3e_with_url where dist < 20
         ORDER BY cosineDistance(embedding, [{q_emb_str}])
            AS dist 
        LIMIT 3 
        """
# q_str = f"""
#         SELECT document
#            metadata dist
#         FROM default.jd_docs_m3e_with_url
#          ORDER BY L2Distance(embedding, [{q_emb_str}])
#             AS dist {self.dist_order}
#         LIMIT {topk} {' '.join(settings_strs)}
#         """

r = client.query(q_str)

for row in r.result_rows:
    (new_doc, meta, dist) = row
    print(dist)

# 入库 clickhouse 相量值使用 product + tilte + pagecontent，文档存储 document.page_content

In [ ]:
# 加载自定义模块路径
import sys
sys.path.append("..")

In [ ]:
from hashlib import sha1
import logging
from threading import Thread
from typing import Any, Iterable, List, Optional
from langchain.text_splitter import MarkdownHeaderTextSplitter, MarkdownTextSplitter
import langchain_community.vectorstores.clickhouse as clickhouse
from langchain.embeddings import HuggingFaceEmbeddings
from langchain_community.document_loaders import DirectoryLoader
from libs.jd_doc_json_loader import JD_DOC_Loader
import json
logger = logging.getLogger()


def new_add_texts(
    self,
    texts: Iterable[str],
    metadatas: Optional[List[dict]] = None,
    batch_size: int = 32,
    ids: Optional[Iterable[str]] = None,
    **kwargs: Any,
) -> List[str]:
    """Insert more texts through the embeddings and add to the VectorStore.

    Args:
        texts: Iterable of strings to add to the VectorStore.
        ids: Optional list of ids to associate with the texts.
        batch_size: Batch size of insertion
        metadata: Optional column data to be inserted

    Returns:
        List of ids from adding the texts into the VectorStore.

    """
    # Embed and create the documents
    ids = ids or [sha1(t.encode("utf-8")).hexdigest() for t in texts]
    colmap_ = self.config.column_map
    transac = []
    # column_names = {
    #     colmap_["id"]: ids,
    #     colmap_["document"]: texts,
    #     colmap_["embedding"]: self.embedding_function.embed_documents(list(texts)),
    # }

    idx = 0
    new_texts = []
    for doc in texts:
        new_text = metadatas[idx]["product"]+"--"+metadatas[idx]["title"]
        new_text = new_text+doc
        new_texts.append(new_text)
        idx += 1

    column_names = {
        colmap_["id"]: ids,
        colmap_["document"]: texts,
        colmap_["embedding"]: self.embedding_function.embed_documents(new_texts),
    }
    metadatas = metadatas or [{} for _ in texts]

    column_names[colmap_["metadata"]] = map(json.dumps, metadatas)
    assert len(set(colmap_) - set(column_names)) >= 0
    keys, values = zip(*column_names.items())
    try:
        t = None
        for v in self.pgbar(
            zip(*values), desc="Inserting data...", total=len(metadatas)
        ):
            assert (
                len(v[keys.index(self.config.column_map["embedding"])]) == self.dim
            )
            transac.append(v)
            if len(transac) == batch_size:
                if t:
                    t.join()
                t = Thread(target=self._insert, args=[transac, keys])
                t.start()
                transac = []
        if len(transac) > 0:
            if t:
                t.join()
            self._insert(transac, keys)
        return [i for i in ids]
    except Exception as e:
        logger.error(
            f"\033[91m\033[1m{type(e)}\033[0m \033[95m{str(e)}\033[0m")
        return []


root_dir = "/root/jd_docs"
loader = DirectoryLoader(
    '/root/jd_docs', glob="**/*.json", loader_cls=JD_DOC_Loader)
docs = loader.load()
model_kwargs = {"device": "cuda"}
embeddings = HuggingFaceEmbeddings(
    model_name="/root/models/moka-ai-m3e-large", model_kwargs=model_kwargs)

headers_to_split_on = [
    ("#", "#"),
    ("##", "##"),
    ("###", "###"),
    ("####", "####"),
    ("#####", "#####"),
]
MarkdownTextSplitter
markdown_splitter = MarkdownHeaderTextSplitter(
    headers_to_split_on=headers_to_split_on)

new_docs = []
batch = 200

settings = clickhouse.ClickhouseSettings(
    table="jd_docs_m3e_with_url_splited", username="default", password="Git785230", host="10.0.1.94")

count = 0
# 将 product tilte 作为正文加入document.page_content,分批次入库
for doc in docs:
    chunked_docs = markdown_splitter.split_text(doc.page_content)
    # head = doc.metadata["product"]+"--"+doc.metadata["title"]+"\n"
    for chunded_doc in chunked_docs:
        # doc_head = head
        doc_head = ""
        if len(chunded_doc.metadata.items()) > 0:
            for k, v in chunded_doc.metadata.items():
                doc_head = doc_head+k+v+"\n"

        content = doc_head+chunded_doc.page_content
        new_doc = doc
        new_doc.page_content = content
        count += 1
        new_docs.append(new_doc)

        if len(new_docs) > batch:
            texts = [d.page_content for d in new_docs]
            metadatas = [d.metadata for d in new_docs]
            clickhouse.Clickhouse.add_texts = new_add_texts
            clickhouse.Clickhouse.from_texts(
                texts, embeddings, metadatas, config=settings)
            # _ = clickhouse.Clickhouse.from_documents(
            #     new_docs, embeddings, config=settings)
            new_docs.clear()
# if len(new_docs) > 0:
#     _ = clickhouse.Clickhouse.from_documents(
#         new_docs, embeddings, config=settings)
print(count)

## 使用qdrant向量库

In [ ]:
from langchain_qdrant import Qdrant
from langchain.embeddings import HuggingFaceEmbeddings
model_kwargs = {"device": "cuda"}
embeddings = HuggingFaceEmbeddings(
    model_name="/root/models/moka-ai-m3e-large", model_kwargs=model_kwargs)

url = "http://10.0.1.94:6333"
qdrant = Qdrant.from_documents(
    docs,
    embeddings,
    url=url,
    prefer_grpc=True,
    collection_name="my_documents",
)

In [ ]:
from typing import List
from langchain_qdrant import Qdrant
from langchain.embeddings import HuggingFaceEmbeddings

model_kwargs = {"device": "cuda"}
embeddings = HuggingFaceEmbeddings(
    model_name="/root/models/moka-ai-m3e-large", model_kwargs=model_kwargs)


url = "http://10.0.1.94:6333"

qdrant = Qdrant.from_existing_collection(
    embedding=embeddings,
    collection_name="my_documents",
    url="http://10.0.1.94:6333",
)

In [ ]:

import re
txt = r"\u003C\u002Fpre\u003E\u003C\u002Fdiv\u003E\u003Cp\u003E等待一段时间后，执行kubectl logs gpu-pod，并参考下图验证输出结果。\u003C\u002Fp\u003E"
txt_not = r"afdsf等待一段时间后，执行kubectl logs gpu-pod，并参考下图验证输出结"


def include_unicode(text):
    match = r"\\[uU]([0-9a-fA-F]{4})"
    if re.search(r"\\[uU]([0-9a-fA-F]{4})", text):
        return True
    else:
        return False


def start_with_unicode(text):
    match = r"^\\[uU]([0-9a-fA-F]{4})"
    if re.search(match, text):
        return True
    else:
        return False


# include_unicode(txt_not)
start_with_unicode(txt)